In [1]:
import os
from chains.local_doc_qa import LocalDocQA
from configs.common_config import *
import nltk
from loader.models.base import (BaseAnswer,
                         AnswerResult,
                         AnswerResultStream,
                         AnswerResultQueueSentinelTokenListenerQueue)
import loader.models.shared as shared
from loader.models.loader.args import parser
from loader.models.loader import LoaderCheckPoint
import time
import datetime
import numpy as np


embedding_model_dict_list = list(embedding_model_dict.keys())
print(embedding_model_dict_list)
llm_model_dict_list = list(llm_model_dict.keys())
print(llm_model_dict_list)

local_doc_qa = LocalDocQA()
local_doc_qa.chunk_content = False # 这里控制结果是否拼接
local_doc_qa.chunk_size = 250
if local_doc_qa.embeddings is None:
    local_doc_qa.init_embedding()


vs_path = '/mnt/workspace/pangtianqi/medical_kb_chatbot/vector_store/test_1'

if local_doc_qa.vector_store is None:
    print('初始化加载向量库')
    local_doc_qa.init_vector_store(vs_path)

vector_store = local_doc_qa.vector_store
index = vector_store.index.ntotal
print(index)
# embedding_layer = local_doc_qa.embeddings()
# embedding_output = embedding_layer(local_doc_qa.vector_store)
# print(embdedding_output)

INFO  2023-09-12 06:09:13,120-1d: 
loading model config
llm device: cuda
embedding_model:m3e-base #这一行自己加的
embedding device: cuda
dir: /mnt/workspace/pangtianqi/medical_kb_chatbot
flagging username: 4f850e9518b14add8f069877aba43e6d



[2023-09-12 06:09:16,204] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/pai/envs/kb_chat/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/pai/envs/kb_chat/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/pai/envs/kb_chat/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...
['ernie-tiny', 'ernie-base', 'text2vec-base', 'text2vec-large-chinese', 'm3e-small', 'm3e-base']
['PULSE']
初始化加载向量库
zhelima?
Converting the loaded index to IndexFlatIP...
index维度 768
docstore <langchain.do

In [16]:
#query = '经常用阿奇霉素会有什么副作用吗？'
query = '氯雷他定片的注意事项？'
# related_docs_with_score = local_doc_qa.vector_store.similarity_search_with_score(query, k=10)
# test_query_1 = vector_store.embedding_function(query)
# test_query_1 = np.array(test_query_1)
# vector_store.index.add(test_query_1)
# index = vector_store.index.ntotal
# print(index)
# print(test_query_1)
related_docs_with_score = local_doc_qa.vector_store.similarity_search_with_score_by_vector(query,k=20)
print(related_docs_with_score)
# responses = local_doc_qa.get_base_answer(model name = ,query=query, prompt_template=[])

12321321313 [[ 4.33323354e-01  6.84538841e-01 -8.28642175e-02  1.28718078e-01
  -7.11266577e-01 -7.41127431e-01 -3.45555305e-01  1.01130620e-01
  -6.67962015e-01  3.08332741e-01  2.07875788e-01  5.25052428e-01
   1.72321931e-01 -7.06247449e-01 -7.36676455e-01 -4.53792781e-01
   3.44389021e-01  6.97834730e-01  1.11203618e-01 -8.75333101e-02
  -4.30754684e-02 -6.12118006e-01 -1.17160976e+00  4.98827130e-01
   2.08717093e-01  9.61757064e-01  3.11147004e-01 -3.37086730e-02
  -2.40229398e-01  5.53929687e-01  8.03187907e-01 -5.91213942e-01
   2.16665357e-01  7.81935096e-01  7.27913201e-01 -3.59024145e-02
   4.14987057e-01  6.94630817e-02  2.60069966e-02  1.12354887e+00
   3.18247110e-01  1.02343571e+00 -1.08266659e-01 -1.20806262e-01
   6.79958045e-01 -1.15095779e-01  5.11662126e-01  5.71968734e-01
  -5.84138453e-01  2.45300263e-01 -8.12190890e-01  7.04562855e+00
  -6.86941743e-01  1.13102770e+00 -7.44345710e-02  1.01468992e+00
  -2.77759701e-01  2.72440374e-01  2.89334506e-01  5.69694936e-0

In [17]:
from langchain.docstore.document import Document
# 假设你的数据存储在一个名为 documents 的列表中
documents = related_docs_with_score
# 创建一个新的列表来存储处理后的数据
processed_data = []
# 遍历每个 Document 对象，提取所需字段并添加查询
for doc in documents:
    content = doc.page_content
    metadata = doc.metadata
# 构建新的数据结构，包括查询、page_content 和 metadata
    processed_doc = {
       'query': query,
       'page_content': content,
       #'metadata': metadata
    }
# 将处理后的数据添加到列表
    processed_data.append(processed_doc)
processed_data

[{'query': '氯雷他定片的注意事项？',
  'page_content': '氯雷他定片 注意事项 1.严重肝功能不全的患者请在医生指导下使用。\n2.妊娠期及哺乳期妇女慎用。\n3.在作皮试前的约48小时左右应中止使用本品，因抗组胺药能阻止或降低皮试的阳性反应发生。(皮试是皮肤(或皮内)敏感试验的简称。某些药物在临床使用过程中容易发生过敏反应，如青霉素、链霉素、细胞色素C等，常见的过敏反应包括皮疹、荨麻疹、皮炎、发热、血管神经性水肿、哮喘、过敏性休克等，其中以过敏性休克最为严重，甚至可导致死亡。)\n4.对本品过敏者禁用，过敏体质者慎用。\n5.本品性状发生改变时禁止使用。\n6.请将本品放在儿童不能接触的地方。\n7.儿童必须在成人监护下使用。\n8.如正在使用其他药品，使用本品前请咨询医师或药师。 商品名: 开瑞坦'},
 {'query': '氯雷他定片的注意事项？',
  'page_content': '氯雷他定胶囊 注意事项 1\xa0精神运动试验研究表明，本品与酒同时服用时不会产生药力相加作用；2\xa0抗组胺药能清除或减轻皮肤对所有变应原的阳性反应，因此在作皮试前约48小时应停止使用氯雷他定；3\xa0本品对心脏功能无影响，但偶有心律失常报道，有心律失常病史者应慎用；4\xa0对肝功能不全者，消除半衰期有所延长，请在医生指导下使用，可按10mg/次，隔日1次服药。5\xa0肾功能不全者慎用。 商品名: 星元佳 海王抒瑞'},
 {'query': '氯雷他定片的注意事项？',
  'page_content': '氯雷他定片 禁忌症 对本品中的成分过敏或特异体质的病人禁用。 商品名: 开瑞坦'},
 {'query': '氯雷他定片的注意事项？',
  'page_content': '氯雷他定片 儿童禁忌 12岁以下儿童应用本品的安全性尚未确定。 商品名: 开瑞坦'},
 {'query': '氯雷他定片的注意事项？',
  'page_content': '氯雷他定片 药物相互作用 1.同时服用酮康唑、大环内酯类抗生素、西咪替丁、茶碱等药物，会提高氯雷他定在血浆中的浓度，应慎用。其他已知能抑制肝脏代谢的药物，在未明确与氯雷他定相互作用前应谨慎合用。\n2.抑制肝药物代谢酶功能的药物能使本品的代谢减慢。每日同服酮康唑400mg，可使氯

In [4]:
#先将上一步输出进行tokenizer再embedding
from transformers import AutoTokenizer
model_name = '/mnt/workspace/pangtianqi/LLM_MODEL/chatglm-6b'
#初始化tokenizer model
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True)

In [5]:
#将embedding后的数据再输入到GLU层得到特征之间更为复杂的关系
import torch
from torch import nn
from configuration_chatglm import ChatGLMConfig
from torch.nn import LayerNorm
from chatglm1 import *
# 对每个数据项进行编码
for item in processed_data:
    query = item['query']
    page_content = item['page_content']
    # 使用tokenizer编码文本
    encoded = tokenizer(query, page_content, padding=True, truncation=True, return_tensors='pt')
    print(encoded['input_ids'].shape)
    # print("encoded",encoded)



Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


torch.Size([1, 24])
torch.Size([1, 72])
torch.Size([1, 198])
torch.Size([1, 24])
torch.Size([1, 30])
torch.Size([1, 131])
torch.Size([1, 32])
torch.Size([1, 92])
torch.Size([1, 26])
torch.Size([1, 31])
torch.Size([1, 112])
torch.Size([1, 69])
torch.Size([1, 45])
torch.Size([1, 23])
torch.Size([1, 74])
torch.Size([1, 25])
torch.Size([1, 23])
torch.Size([1, 24])
torch.Size([1, 24])
torch.Size([1, 23])


In [6]:
configuration = ChatGLMConfig(prefix_projection=True,pre_seq_len=130528,num_layers=4)
embedding = PrefixEncoder(configuration)
#print(embedding)
layer_norm = LayerNorm((16384,),eps=1e-05,elementwise_affine=True)
#print(layer_norm)
for i in range(0,20):
    # input_tensor = torch.tensor(encoded_query["input_ids"], dtype=torch.long)
    # print(input_tensor)
    embedded_output = embedding(encoded["input_ids"])
    # print("embedded_output",embedded_output)
    norm_output = layer_norm(embedded_output)

In [13]:
for i in range(0,20):
    size = norm_output.shape
    value = size[-1]
    glu_instance = GLU(hidden_size=value,activation_func = gelu)
    output = glu_instance(norm_output)
   #  final_output = layer_norm(output)
    print(output)
    # has_nan = torch.isnan(final_output).any() 
#     if has_nan:
#     # 筛除包含 NaN 的张量
#        new_output = final_output[~torch.isnan(final_output)]
#     else:
#     # 没有 NaN，将 final_output 直接赋值给 new_output
#        new_output = final_output
# print(new_output)    

tensor([[[-1.2629e+19,  4.5884e-41, -1.2629e+19,  ...,  0.0000e+00,
           4.2653e+34,  0.0000e+00],
         [-1.2629e+19,  4.5884e-41, -1.2629e+19,  ...,  0.0000e+00,
           4.2653e+34,  0.0000e+00],
         [-1.2629e+19,  4.5884e-41, -1.2629e+19,  ...,  0.0000e+00,
           4.2653e+34,  0.0000e+00],
         ...,
         [-1.2629e+19,  4.5884e-41, -1.2629e+19,  ...,  0.0000e+00,
           4.2653e+34,  0.0000e+00],
         [-1.2629e+19,  4.5884e-41, -1.2629e+19,  ...,  0.0000e+00,
           4.2653e+34,  0.0000e+00],
         [-1.2629e+19,  4.5884e-41, -1.2629e+19,  ...,  0.0000e+00,
           4.2653e+34,  0.0000e+00]]], grad_fn=<ViewBackward0>)
tensor([[[ 1.1874e+35,  0.0000e+00, -1.2629e+19,  ..., -8.2722e-01,
          -4.6972e-02,  2.7723e-01],
         [ 1.1874e+35,  0.0000e+00, -1.2629e+19,  ..., -8.2722e-01,
          -4.6972e-02,  2.7723e-01],
         [ 1.1874e+35,  0.0000e+00, -1.2629e+19,  ..., -8.2722e-01,
          -4.6972e-02,  2.7723e-01],
         ...,


In [8]:
#  # 请替换为正确的模块和类的导入路径
# from transformers import BatchEncoding
# configuration = ChatGLMConfig(prefix_projection=True,pre_seq_len=130528,num_layers=2)
# # print(configuration)
# embedding = PrefixEncoder(configuration)
# print(embedding)
# batch_encoding = BatchEncoding()
# batch_encoding["input_ids"] = [sample["input_ids"] for sample in encoded_data]
# batch_encoding["attention_mask"] = [sample["attention_mask"] for sample in encoded_data]
# print(batch_encoding)
# # batch_encoding["input_ids"] = encoded_data
# # 将列表转换为torch.Tensor
# input_tensor = torch.tensor(batch_encoding["input_ids"], dtype=torch.long)
# # 调用PrefixEncoder的forward方法进行嵌入
# embedded_output = embedding(input_tensor)
# # 打印嵌入后的输出形状
# print("Embedded Output Shape:", embedded_output.shape)


In [9]:
#加入norm层
# from torch.nn import LayerNorm
# layer_norm = LayerNorm((16384,),elementwise_affine=True)
# print(layer_norm)
# final_output = layer_norm(embedded_output)
# transposed_output = torch.transpose(embedded_output, 1, 2)
# transposed_output.shape

In [10]:
# from chatglm1 import *
# glu_instance = GLU(hidden_size=16,activation_func = gelu)
# print(glu_instance)
# output = glu_instance(transposed_output)
# output

In [11]:
#把output排序，然后得到前5个，然后再经过一个decoder解码得到输出

